In [10]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM

In [11]:
# Loading data
measures = pd.read_csv("../data/measures_database.csv", sep=';', encoding='utf8')
stock_it = pd.read_csv("../data/FTSE_MIB_large.csv", sep='\t', encoding='utf7')

In [12]:
# Cleaning
measures = measures.dropna(axis='rows', subset=['DATE_IMPLEMENTED'])
measures = measures.dropna(axis='columns', how='all')
measures['DATE_IMPLEMENTED'] = pd.to_datetime(measures['DATE_IMPLEMENTED'], dayfirst=True)

measures = measures[measures['LOG_TYPE'] != "Phase-out measure"] # Not looking at phasing out atm

measures.MEASURE = measures['MEASURE'].str.split().str.join(' ') # remove HTML \xa0
measures = measures[measures['MEASURE'] != 'Strengthening the public health system']
measures = measures[measures['MEASURE'] != 'Economic measures']
measures = measures[measures['MEASURE'] != 'Awareness campaigns']
measures = measures[measures['MEASURE'] != 'Psychological assistance and medical social work']
measures = measures[measures['MEASURE'] != 'Changes in prison-related policies']
measures = measures[measures['MEASURE'] != 'Other public health measures enforced']

measures.sort_values(by='DATE_IMPLEMENTED',ascending=True, inplace=True)


stock_it['Date'] = pd.to_datetime(stock_it['Date'])
stock_it.sort_values(by='Date',ascending=True, inplace=True)
stock_it.reset_index(inplace=True)

In [13]:
# Selecting
country_name = 'Italy'

measures_it = measures[measures['COUNTRY'] == country_name]
measures_it.reset_index(inplace=True)

In [15]:
# Get subset needed
measure_w_date = measures_it[['DATE_IMPLEMENTED', 'MEASURE']]
stock_w_date = stock_it[["Date", "Price"]]

In [19]:
#setting index
stock_w_date.index = stock_w_date.Date
stock_w_date.drop('Date', axis=1, inplace=True)

AttributeError: 'DataFrame' object has no attribute 'Date'

In [20]:
# Creating train and test sets
dataset = stock_w_date.values

pion

train = dataset[0:,:]
valid = dataset[987:,:]

#converting dataset into x_train and y_train
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(dataset)

x_train, y_train = [], []
for i in range(60,len(train)):
    x_train.append(scaled_data[i-60:i,0])
    y_train.append(scaled_data[i,0])
x_train, y_train = np.array(x_train), np.array(y_train)

x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1))

# create and fit the LSTM network
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1],1)))
model.add(LSTM(units=50))
model.add(Dense(1))

model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(x_train, y_train, epochs=1, batch_size=1, verbose=2)

#predicting 246 values, using past 60 from the train data
inputs = stock_w_date[len(stock_w_date) - len(valid) - 60:].values
inputs = inputs.reshape(-1,1)
inputs  = scaler.transform(inputs)

X_test = []
for i in range(60,inputs.shape[0]):
    X_test.append(inputs[i-60:i,0])
X_test = np.array(X_test)

X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))
closing_price = model.predict(X_test)
closing_price = scaler.inverse_transform(closing_price)

Train on 276 samples
276/276 - 7s - loss: 0.0225


IndexError: tuple index out of range

In [ ]:
train = new_data[:987]
valid = new_data[987:]
valid['Predictions'] = closing_price
plt.plot(train['Close'])
plt.plot(valid[['Close','Predictions']])